In [1]:
from shared_code.utility.schemas.spark_table_schema import image_table_schema,image_table_schema_with_caption_tokens
from shared_code.utility.spark.set_environ import *
from shared_code.utility.storage.blob import BlobAdapter
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

set_azure_env()

spark = get_session('cloud-mover')

In [2]:
spark.read.parquet("D:\\data\\processed\\reddit_images_processed_with_caption_tokens.parquet", schema=image_table_schema_with_caption_tokens).createOrReplaceTempView("reddit_images_processed_with_caption_tokens")

existing_images_not_curated = spark.sql("""
select * from
reddit_images_processed_with_caption_tokens
where small_image is not null
and small_image != ''
and small_image != 'None'
and small_image != 'Nan'
and updated_caption != 'None'
and exists
and not curated
""")

existing_images_not_curated.limit(10).toPandas()

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated,caption_tokens,text_tokens
0,training,110sqfl,D:\data\images\SFWNextDoorGirls\gwi4xae6pvha1.jpg,Ready for the big game❤️💚,110sqfl,TaylorRae30,https://i.redd.it/gwi4xae6pvha1.jpg,NaN,/r/SFWNextDoorGirls/comments/110sqfl/ready_for...,e96bc0f370ea29f2aa404fde5de604a8,SFWNextDoorGirls,A woman in a pink dress is talking to a man in...,True,gwi4xae6pvha1.jpg,a woman holding a bottle in her mouth,D:\data\images\SFWNextDoorGirls\thumbnail\gwi4...,False,"[(woman, NN), (holding, VBG), (bottle, NN), (i...","[(Ready, JJ), (for, IN), (the, DT), (big, JJ),..."
1,training,110sxa0,D:\data\images\SFWNextDoorGirls\athgu0lpqvha1.jpg,"Pretending to be shy, did I succeed? 😜",110sxa0,texassageember,https://i.redd.it/athgu0lpqvha1.jpg,NaN,/r/SFWNextDoorGirls/comments/110sxa0/pretendin...,08ab1dfc840e049b9bb73eb3158b1c4e,SFWNextDoorGirls,A girl in a pink shirt and blue jeans is talki...,True,athgu0lpqvha1.jpg,a woman in a red dress is smiling,D:\data\images\SFWNextDoorGirls\thumbnail\athg...,False,"[(woman, NN), (in, IN), (red, JJ), (dress, NN)...","[(Pretending, VBG), (to, TO), (be, VB), (shy, ..."
2,training,110sxyb,D:\data\images\Faces\fouhu3hb9uha1.jpg,My ass knows what it wants and it wants you. ;),110sxyb,ResoluteGlitter,https://i.redd.it/fouhu3hb9uha1.jpg,Self,/r/Faces/comments/110sxyb/my_ass_knows_what_it...,4dc3258bf974e0925e2a9ffa72cc368e,Faces,A woman in a white dress is talking to a man i...,True,fouhu3hb9uha1.jpg,a woman wearing glasses and a black shirt,D:\data\images\Faces\thumbnail\fouhu3hb9uha1.jpg,False,"[(woman, NN), (wearing, VBG), (glasses, NNS), ...","[(My, PRP$), (ass, NN), (knows, VBZ), (what, W..."
3,training,110t034,D:\data\images\CityPorn\lismgrhcrvha1.jpg,"Washington, the Bogota of Colombia",110t034,Bloxburgian1945,https://i.redd.it/lismgrhcrvha1.jpg,NaN,/r/CityPorn/comments/110t034/washington_the_bo...,4bdb5ebed7bbc0954be55f8c49f9191f,CityPorn,A man in a blue shirt is sitting on the side o...,True,lismgrhcrvha1.jpg,a man and woman standing next to each other ne...,D:\data\images\CityPorn\thumbnail\lismgrhcrvha...,False,"[(man, NN), (and, CC), (woman, NN), (standing,...","[(Washington, NNP), (the, DT), (Bogota, NNP), ..."
4,training,110t2ax,D:\data\images\Faces\gMnFY6t.jpg,😛,110t2ax,postalTimalia716,https://i.imgur.com/gMnFY6t.jpg,Self,/r/Faces/comments/110t2ax/_/,d54a462a3fba88657de72d3a719114b5,Faces,A girl in a pink shirt and blue jeans is smoki...,True,gMnFY6t.jpg,a woman in a white dress holding a white object,D:\data\images\Faces\thumbnail\gMnFY6t.jpg,False,"[(woman, NN), (in, IN), (white, JJ), (dress, N...",[]
5,training,110t3eq,D:\data\images\Faces\xi6cfzkkauha1.jpg,F18.. Be honest...,110t3eq,SockBeneficial2643,https://i.redd.it/xi6cfzkkauha1.jpg,Self,/r/Faces/comments/110t3eq/f18_be_honest/,9d7c9f37659f71e82b5cdb49b9badfc6,Faces,A girl in a pink dress is smiling and holding ...,True,xi6cfzkkauha1.jpg,a woman with long hair is wearing a pink wig,D:\data\images\Faces\thumbnail\xi6cfzkkauha1.jpg,False,"[(woman, NN), (with, IN), (long, JJ), (hair, N...","[(.., RB), (Be, NNP), (honest, JJS), (..., :)]"
6,training,110t52t,D:\data\images\SFWRedheads\xuf4kk4hsvha1.jpg,Fansly coming soon...,110t52t,EmoryJade777,https://i.redd.it/xuf4kk4hsvha1.jpg,NaN,/r/SFWRedheads/comments/110t52t/fansly_coming_...,abbc50da58591e7afa7c52316fe89a30,SFWRedheads,A girl in a pink dress is smiling.,True,xuf4kk4hsvha1.jpg,a woman is looking at her reflection in the mi...,D:\data\images\SFWRedheads\thumbnail\xuf4kk4hs...,False,"[(woman, NN), (is, VBZ), (looking, VBG), (at, ...","[(Fansly, RB), (coming, VBG), (soon, RB), (......"
7,training,110t5c2,D:\data\images\Faces\kqs5yy60buha1.jpg,Mandatory Elevator Selfie😂,110t5c2,itsmeSexyLove,https://i.redd.it/kqs5yy60buha1.jpg,Self,/r/Faces/comments/110t5c2/mandatory_elevator_s...,6f43221a106c1276d8dca07caf3365f7,Faces,A woman in a white shirt and blue jeans is tal..

In [3]:
foo = existing_images_not_curated.rdd.map(lambda x: ({'image_path': x['small_image'], 'name': x['image_name'], 'sub':x['subreddit']})).collect()

In [6]:
blob_container_name = 'unprocessed'
blob_adapter = BlobAdapter(blob_container_name)

container_client = blob_adapter.get_container_client()

present_containers = container_client.list_containers()

total_items = len(foo)
print(f"Total items: {total_items}")
for elem in foo:
    if total_items % 100 == 0:
        print(f"Remaining: {total_items}")
    data = elem['image_path']
    with open(data, "rb") as f:
        data = f.read()
        out_path = f"{elem['sub']}/{elem['name']}"
        blob_adapter.upload_blob(data=data, blob_name=out_path)
        total_items -= 1

Total items: 4638
Remaining: 4600
Remaining: 4500
Remaining: 4400
Remaining: 4300
Remaining: 4200
Remaining: 4100
Remaining: 4000
Remaining: 3900
Remaining: 3800
Remaining: 3700
Remaining: 3600
Remaining: 3500
Remaining: 3400
Remaining: 3300
Remaining: 3200
Remaining: 3100
Remaining: 3000
Remaining: 2900
Remaining: 2800
Remaining: 2700
Remaining: 2600
Remaining: 2500
Remaining: 2400
Remaining: 2300
Remaining: 2200
Remaining: 2100
Remaining: 2000
Remaining: 1900
Remaining: 1800
Remaining: 1700
Remaining: 1600
Remaining: 1500
Remaining: 1400
Remaining: 1300
Remaining: 1200
Remaining: 1100
Remaining: 1000
Remaining: 900
Remaining: 800
Remaining: 700
Remaining: 600
Remaining: 500
Remaining: 400
Remaining: 300
Remaining: 200
Remaining: 100


In [12]:
foo = spark.createDataFrame(foo).toPandas()
foo.to_csv("D:\\data\\processed\\manifest.csv", index=False)
with open("D:\\data\\processed\\manifest.csv", "rb") as f:
    data = f.read()
    blob_adapter.upload_blob(data=data, blob_name="manifest.csv")

D:\workspaces\General\venv\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
D:\workspaces\General\venv\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
